In [ ]:
import torch
from typing import Tuple
from torch import Tensor
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torch.utils.data import dataset
import math
import torchtext
from tqdm import tqdm

In [ ]:
pip install torchdata

In [ ]:
from torchtext.datasets import PennTreebank 

In [ ]:
train_iter = PennTreebank(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = torchtext.vocab.build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [ ]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
  """Converts raw text into a flat Tensor."""
  data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
  return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

In [ ]:
train_iter, val_iter, test_iter = PennTreebank()
train_data= data_process(train_iter)
val_data= data_process(val_iter)
test_data= data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

In [ ]:
batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

In [ ]:
print(train_data[0])

tensor([9892,    4,   31,   16, 1925,  178,    3, 2295,    5,   42, 3412, 1611,
           1,  730, 3722, 5722, 2124, 1678,  796,  829], device='cuda:0')


In [ ]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [ ]:
class LSTM(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, tie_weights):
    super().__init__()

    self.num_layers = num_layers
    self.hidden_dim = hidden_dim
    self.embedding_dim = embedding_dim

    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, 
                    batch_first=True)
    self.linear = nn.Linear(hidden_dim, vocab_size)

    if tie_weights:
          assert embedding_dim == hidden_dim, 'cannot tie, check dims'
          self.linear.weight = self.embedding.weight
    self.init_weights()

  def forward(self, x):
    # x is a batch of input sequences
    x = self.embedding(x)
    x, _ = self.lstm(x)
    x = self.linear(x)
    return x

  def init_weights(self):
    init_range_emb = 0.1
    init_range_other = 1/math.sqrt(self.hidden_dim)
    self.embedding.weight.data.uniform_(-init_range_emb, init_range_emb)
    self.linear.weight.data.uniform_(-init_range_other, init_range_other)
    self.linear.bias.data.zero_()
    for i in range(self.num_layers):
        self.lstm.all_weights[i][0] = torch.FloatTensor(self.embedding_dim,
                self.hidden_dim).uniform_(-init_range_other, init_range_other) 
        self.lstm.all_weights[i][1] = torch.FloatTensor(self.hidden_dim, 
                self.hidden_dim).uniform_(-init_range_other, init_range_other)
                 

In [ ]:
vocab_size = len(vocab)
embedding_dim = 200
hidden_dim = 200
num_layers = 2
tie_weights = True
model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, tie_weights)

In [ ]:
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        #seq_len = data.size(0)
        output = model(data)
        loss = criterion(output.view(-1, vocab_size), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

In [ ]:
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            output = model(data)
            output_flat = output.view(-1, vocab_size)
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [ ]:
best_val_loss = float('inf')
epochs = 3
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()

| epoch   1 |   200/26411 batches | lr 5.00 | ms/batch 61.93 | loss  7.06 | ppl  1163.18
| epoch   1 |   400/26411 batches | lr 5.00 | ms/batch 60.25 | loss  6.59 | ppl   724.39
| epoch   1 |   600/26411 batches | lr 5.00 | ms/batch 59.65 | loss  6.48 | ppl   652.72
| epoch   1 |   800/26411 batches | lr 5.00 | ms/batch 59.92 | loss  6.28 | ppl   532.78
| epoch   1 |  1000/26411 batches | lr 5.00 | ms/batch 60.27 | loss  6.16 | ppl   472.22
| epoch   1 |  1200/26411 batches | lr 5.00 | ms/batch 60.87 | loss  6.07 | ppl   433.82
| epoch   1 |  1400/26411 batches | lr 5.00 | ms/batch 60.76 | loss  6.08 | ppl   435.98
| epoch   1 |  1600/26411 batches | lr 5.00 | ms/batch 64.71 | loss  5.99 | ppl   400.76
| epoch   1 |  1800/26411 batches | lr 5.00 | ms/batch 59.89 | loss  5.99 | ppl   400.10
| epoch   1 |  2000/26411 batches | lr 5.00 | ms/batch 60.65 | loss  5.85 | ppl   348.62
| epoch   1 |  2200/26411 batches | lr 5.00 | ms/batch 60.17 | loss  5.67 | ppl   289.36
| epoch   1 |  2400/2

In [ ]:
test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| End of training | test loss  4.94 | test ppl   140.42
